In [ ]:
import os
import re
import openai
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# OpenAI API 키 설정
openai.api_key = ""

# 1. 문서 로드 및 텍스트 병합
file_path = "./한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf"

if not os.path.exists(file_path):
    raise FileNotFoundError(f"File not found: {file_path}")

loader = PyPDFLoader(file_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30
)
chunks = text_splitter.split_documents(documents)

# 모든 청크를 병합
all_text = " ".join([
    re.sub(r'\s+', ' ', re.sub(r'[^\w\s.]', '', chunk.page_content)).strip()
    for chunk in chunks
])

# OpenAI GPT API를 이용한 검색
async def query_with_gpt_cancer(question):
    try:
        response = await openai.ChatCompletion.acreate(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant for insurance document analysis."},
                {"role": "user", "content": f"Context:\n{all_text}\n\nQuestion: {question}\n\nAnswer:"}
            ]
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"오류 발생: {e}"


In [ ]:
# OpenAI API 키는 이전과 동일
file_path = "./한화 손해보험.pdf"

if not os.path.exists(file_path):
    raise FileNotFoundError(f"File not found: {file_path}")

loader = PyPDFLoader(file_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30
)
chunks = text_splitter.split_documents(documents)

# 모든 청크를 병합
all_text_life = " ".join([
    re.sub(r'\s+', ' ', re.sub(r'[^\w\s.]', '', chunk.page_content)).strip()
    for chunk in chunks
])

# OpenAI GPT API를 이용한 검색 (손해보험용)
async def query_with_gpt_life(question):
    try:
        response = await openai.ChatCompletion.acreate(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant for insurance document analysis."},
                {"role": "user", "content": f"Context:\n{all_text_life}\n\nQuestion: {question}\n\nAnswer:"}
            ]
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"오류 발생: {e}"


In [ ]:
file_path = "./한화 개인용 자동차보험.pdf"

if not os.path.exists(file_path):
    raise FileNotFoundError(f"File not found: {file_path}")

loader = PyPDFLoader(file_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30
)
chunks = text_splitter.split_documents(documents)

# 모든 청크를 병합
all_text_car = " ".join([
    re.sub(r'\s+', ' ', re.sub(r'[^\w\s.]', '', chunk.page_content)).strip()
    for chunk in chunks
])

# OpenAI GPT API를 이용한 검색 (자동차보험용)
async def query_with_gpt_car(question):
    try:
        response = await openai.ChatCompletion.acreate(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant for insurance document analysis."},
                {"role": "user", "content": f"Context:\n{all_text_car}\n\nQuestion: {question}\n\nAnswer:"}
            ]
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"오류 발생: {e}"


In [ ]:
import gradio as gr

# RAG 체인 정의
async def query_with_gpt(question, context):
    try:
        response = await openai.ChatCompletion.acreate(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant for analyzing insurance documents."},
                {"role": "user", "content": f"Context: {context}\n\nQuestion: {question}\nAnswer:"}
            ]
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"오류 발생: {str(e)}"

# 선택된 체인 관리
selected_chain = {"current": None}

# 체인 설정 함수
def set_chain(chain_type):
    if chain_type == "생명보험":
        selected_chain["current"] = query_with_gpt_cancer
    elif chain_type == "손해보험":
        selected_chain["current"] = query_with_gpt_life
    elif chain_type == "자동차보험":
        selected_chain["current"] = query_with_gpt_car
    return f"{chain_type} 분석 체인이 선택되었습니다."

# 질문 처리 함수
async def process_question(question):
    if not selected_chain["current"]:
        return "먼저 체인을 선택하세요!"
    chain = selected_chain["current"]
    context = "전체 문서 데이터"  # 체인에 맞는 컨텍스트를 가져옵니다
    return await chain(question)

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("## 보험 문서 분석 GPT 챗봇")

    # 보험 종류 선택
    with gr.Row():
        cancer_btn = gr.Button("생명보험")
        life_btn = gr.Button("손해보험")
        car_btn = gr.Button("자동차보험")

    # 질문 입력 및 출력
    with gr.Row():
        question_input = gr.Textbox(label="질문")
        answer_output = gr.Textbox(label="답변")

    # 버튼 이벤트
    cancer_btn.click(lambda: set_chain("생명보험"), inputs=None, outputs=[answer_output])
    life_btn.click(lambda: set_chain("손해보험"), inputs=None, outputs=[answer_output])
    car_btn.click(lambda: set_chain("자동차보험"), inputs=None, outputs=[answer_output])

    # 질문 제출 버튼
    submit_btn = gr.Button("질문 제출")
    submit_btn.click(process_question, inputs=[question_input], outputs=[answer_output])

iface.launch(
    server_name="0.0.0.0",  # 모든 네트워크 인터페이스에서 접근 가능
    server_port=7860,       # 기본 포트 (원하는 경우 변경 가능)
    share=True              # Gradio에서 생성한 공유 링크 활성화
)